In [ ]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

def clean_data(df):
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    # Replace all instances of "[^0-9]" with "" in column: 'Cylinder Count'
    df['Cylinder Count'] = df['Cylinder Count'].str.replace("[^0-9]", "", case=False, regex=True)
    # Replace all instances of "" with "0" in column: 'Cylinder Count'
    df.loc[df['Cylinder Count'].str.lower() == "".lower(), 'Cylinder Count'] = np.nan
    # Drop rows with missing data in column: 'Engine Capacity'
    df = df.dropna(subset=['Engine Capacity'])
    # Drop column: 'Car Status'
    df = df.drop(columns=['Car Status'])
    # Fix parsing error for relative dates
    # Convert 'Date' column to datetime format
    def parse_relative_date(relative_date):
        now = datetime.now()
        try:
            if "hour" in relative_date:
                hours = int(relative_date.split()[0])
                return now - timedelta(hours=hours)
            elif "day" in relative_date:
                days = int(relative_date.split()[0])
                return now - timedelta(days=days)
        except ValueError:
            # Handle invalid formats gracefully
            return now
        return now
    df['Date'] = df['Date'].apply(parse_relative_date)
    # Add 6 days to the parsed dates
    df['Date'] = df['Date'] - timedelta(days=6)
    # Format the date as ddmmyyyy
    df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')
    # Clean Mileage column and rename it
    df['Mileage'] = df['Mileage'].str.replace('KM', '').str.replace(',', '').astype(int)
    df.rename(columns={'Mileage': 'Mileage_in_KM'}, inplace=True)
    # Remove outliers in Mileage_in_KM
    q1 = df['Mileage_in_KM'].quantile(0.25)
    q3 = df['Mileage_in_KM'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df = df[(df['Mileage_in_KM'] >= lower_bound) & (df['Mileage_in_KM'] <= upper_bound)]
    # Drop column: 'Travel Range'
    df = df.drop(columns=['Travel Range'])
    # Drop column: 'Source.Name'
    df = df.drop(columns=['Source.Name'])
    return df

# Loaded variable 'df' from URI: d:\depi final project\Data ContactCars.xlsx
df = pd.read_excel(r'd:\depi final project\Data ContactCars.xlsx')

df_clean = clean_data(df.copy())
df_clean.head()